In [1]:
import numpy as np
import pandas as pd
from sko.GA import GA

## pyomo

In [27]:

import numpy as np
import pyomo.environ as pyo
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import pandas as pd
from amplpy import modules
from typing import List
from pyomo.opt import SolverStatus, TerminationCondition

def preprocess_file(df_params):
    df_params['slope'] = (df_params['sales_at_upper_bound'] - df_params['sales_at_lower_bound']
                          ) / (df_params['price_upper_bound'] - df_params['price_lower_bound'])
    df_params['intercept'] = df_params['sales_at_lower_bound'] - \
        df_params['slope'] * df_params['price_lower_bound']
    df_params['deal_slope'] = (df_params['deal_sales_upper_bound'] - df_params['deal_sales_lower_bound']
                               ) / (df_params['price_upper_bound'] - df_params['price_lower_bound'])
    df_params['deal_intercept'] = df_params['deal_sales_lower_bound'] - \
        df_params['deal_slope'] * df_params['price_lower_bound']
    return df_params

def optimize_with_deal(df_params):
    n_product = len(df_params)
    model = pyo.ConcreteModel()

    def price_bounds_rule(model, i):
        return df_params.loc[i, 'price_lower_bound'], df_params.loc[i, 'price_upper_bound']

    model.prices = pyo.Var(
        range(n_product), domain=pyo.NonNegativeReals, bounds=price_bounds_rule)

    # 每个产品是否做deal
    model.deal = pyo.Var(range(n_product), domain=pyo.NonNegativeIntegers, bounds=(0, 10))

    # set product 1 deal to 1 
    model.deal[0].fix(10.0)

    # 总利润
    def total_profit_rule(model):
        a = df_params['slope']
        b = df_params['intercept']
        a_d = df_params['deal_slope']
        b_d = df_params['deal_intercept']
        total_profit = 0
        for i in range(n_product):
            # 日常每天利润
            fba_commission = model.prices[i] * 0.15
            manual_fee = model.prices[i] * 0.06
            storage_fee = model.prices[i] * 0.02

            # 每个产品的利润 日常
            product_income = model.prices[i] * (1 - df_params['damage_rate'][i]) - df_params['latest_amazon_delivery_fee_usd'][i] - \
                fba_commission - df_params['vat_local'][i] - df_params['promotion_discount'][i] - \
                df_params['ppc_cost'][i] - manual_fee - storage_fee
            product_profit_rmb = product_income * df_params['exchange_rate'][i] - df_params['purchase_cost'][i] - \
                df_params['shipping_and_tax'][i]

            # 每个产品的利润 deal
            product_income_deal = model.prices[i] * 0.85 * (1 - df_params['damage_rate'][i]) - df_params['latest_amazon_delivery_fee_usd'][i] - \
                fba_commission - df_params['vat_local'][i] - df_params['promotion_discount'][i] - \
                df_params['ppc_cost'][i] - manual_fee - storage_fee
            product_profit_rmb_deal = product_income_deal * df_params['exchange_rate'][i] - df_params['purchase_cost'][i] - \
                df_params['shipping_and_tax'][i]

            # 日常销量
            product_sales = a[i] * model.prices[i] + b[i]

            # deal销量
            product_sales_deal = a_d[i] * model.prices[i] + b_d[i]

            total_profit += (30 - model.deal[i]) * product_profit_rmb * product_sales \
                + model.deal[i] * product_profit_rmb_deal * product_sales_deal

        return total_profit

    model.total_profit = pyo.Objective(
        rule=total_profit_rule, sense=pyo.maximize)

    solver = pyo.SolverFactory(modules.find('bonmin'), solver_io='nl')
    result = solver.solve(model)

    if (result.solver.status == SolverStatus.ok ) and (result.solver.termination_condition == TerminationCondition.optimal):
        status = 'optimal'
    elif result.solver.termination_condition == TerminationCondition.infeasible:
        status = 'infeasible'
    else:
        status = result.solver.status

    optimized_prices = [np.round(pyo.value(model.prices[i]),2) for i in range(n_product)]
    deal = [np.round(pyo.value(model.deal[i])).astype(int) for i in range(n_product)]
    max_profit = np.round(pyo.value(model.total_profit),2)
    
    return status, optimized_prices, deal, max_profit

def optimize_without_deal(df_params):
    n_product = len(df_params)
    model = pyo.ConcreteModel()

    def price_bounds_rule(model, i):
        return df_params.loc[i, 'price_lower_bound'], df_params.loc[i, 'price_upper_bound']

    model.prices = pyo.Var(
        range(n_product), domain=pyo.NonNegativeReals, bounds=price_bounds_rule)

    # 总利润
    def total_profit_rule(model):
        a = df_params['slope']
        b = df_params['intercept']
        total_profit = 0
        for i in range(n_product):
            # 日常每天利润
            fba_commission = model.prices[i] * 0.15
            manual_fee = model.prices[i] * 0.06
            storage_fee = model.prices[i] * 0.02

            # 每个产品的利润 日常
            product_income = model.prices[i] * (1 - df_params['damage_rate'][i]) - df_params['latest_amazon_delivery_fee_usd'][i] - \
                fba_commission - df_params['vat_local'][i] - df_params['promotion_discount'][i] - \
                df_params['ppc_cost'][i] - manual_fee - storage_fee
            product_profit_rmb = product_income * df_params['exchange_rate'][i] - df_params['purchase_cost'][i] - \
                df_params['shipping_and_tax'][i]

            # 日常销量
            product_sales = a[i] * model.prices[i] + b[i]

            total_profit += 30 * product_profit_rmb * product_sales

        return total_profit

    model.total_profit = pyo.Objective(
        rule=total_profit_rule, sense=pyo.maximize)

    solver = pyo.SolverFactory(modules.find('bonmin'), solver_io='nl')
    result = solver.solve(model)

    if (result.solver.status == SolverStatus.ok ) and (result.solver.termination_condition == TerminationCondition.optimal):
        status = 'optimal'
    elif result.solver.termination_condition == TerminationCondition.infeasible:
        status = 'infeasible'
    else:
        status = result.solver.status

    optimized_prices = [np.round(pyo.value(model.prices[i]),2) for i in range(n_product)]
    deal = [0 for i in range(n_product)]
    max_profit = np.round(pyo.value(model.total_profit),2)
    
    return status, optimized_prices, deal, max_profit

def result_process(df_params, optimized_prices, deal):
    df_result = df_params.copy()
    df_result['optimized_price'] = optimized_prices
    df_result['expected_sales'] = (
        df_result['slope'] * df_result['optimized_price'] + df_result['intercept']).round(0).astype(int)
    df_result['expected_sales_deal'] = (
        df_result['deal_slope'] * df_result['optimized_price'] + df_result['deal_intercept']).round(0).astype(int)
    df_result['fba_commission_local'] = df_result['optimized_price']*0.15
    df_result['labor_cost_6_percent'] = df_result['optimized_price']*0.06
    df_result['storage_cost_2_percent'] = df_result['optimized_price']*0.02

    # 是否做deal
    df_result['is_deal'] = deal
    df_result['product_income'] = df_result['optimized_price'] * (1 - df_result['damage_rate']) - df_result['latest_amazon_delivery_fee_usd'] - \
        df_result['fba_commission_local'] - df_result['vat_local'] - df_result['promotion_discount'] - \
        df_result['ppc_cost'] - df_result['labor_cost_6_percent'] - \
        df_result['storage_cost_2_percent']
    df_result['product_profit_rmb'] = df_result['product_income'] * df_result['exchange_rate'] - df_result['purchase_cost'] - \
        df_result['shipping_and_tax']
    df_result['product_income_deal'] = df_result['optimized_price'] * 0.85 * (1 - df_result['damage_rate']) - df_result['latest_amazon_delivery_fee_usd'] - \
        df_result['fba_commission_local'] - df_result['vat_local'] - df_result['promotion_discount'] - \
        df_result['ppc_cost'] - df_result['labor_cost_6_percent'] - \
        df_result['storage_cost_2_percent']
    df_result['product_profit_rmb_deal'] = df_result['product_income_deal'] * df_result['exchange_rate'] - df_result['purchase_cost'] - \
        df_result['shipping_and_tax']
    df_result['month_profit_rmb'] = (30 - df_result['is_deal']) * df_result['product_profit_rmb'] * df_result['expected_sales'] \
        + df_result['is_deal'] * df_result['product_profit_rmb_deal'] * df_result['expected_sales_deal']

    df_result = df_result.drop(
        columns=['slope', 'intercept', 'deal_slope', 'deal_intercept'])
    return df_result

In [33]:
df_params = pd.read_json('data/demo_with_deal.json')

dft = preprocess_file(df_params)
results_d = optimize_with_deal(dft)
results_nd = optimize_without_deal(dft)

if results_d[3] > results_nd[3]:  # Comparing max_profit
    results = results_d
else:
    results = results_nd

df_result = result_process(dft, results[1], results[2])
status, optimized_prices, deal, max_profit = results


- 187476.73311710003
- 187482.78771220002
- 187517.54546265 5 = 10
- 187625.80002710002 11=10

In [25]:

optimized_prices_deal = optimized_prices.copy()



dealx = [0 for i in range(len(dft))]
dftr = result_process(dft, optimized_pricesx, dealx)
max_profit_no_deal = dftr.month_profit_rmb.sum()



242126.31270000004

## optimize

In [8]:
import numpy as np
from scipy.optimize import minimize

# 输入数据示例
n_product = len(df_params['slope'])

# 定义总利润函数
def total_profit(x):
    total_profit = 0
    deal_days = x[-n_product:]
    prices = x[:-n_product]
    for i in range(n_product):
        # 日常每天利润
        fba_commission = prices[i] * 0.15
        manual_fee = prices[i] * 0.06
        storage_fee = prices[i] * 0.02

        # 每个产品的利润 日常
        product_income = prices[i] * (1 - df_params['damage_rate'][i]) - df_params['latest_amazon_delivery_fee_usd'][i] - \
            fba_commission - df_params['vat_local'][i] - df_params['promotion_discount'][i] - \
            df_params['ppc_cost'][i] - manual_fee - storage_fee
        product_profit_rmb = product_income * df_params['exchange_rate'][i] - df_params['purchase_cost'][i] - \
            df_params['shipping_and_tax'][i]

        # 每个产品的利润 deal
        product_income_deal = prices[i] * 0.85 * (1 - df_params['damage_rate'][i]) - df_params['latest_amazon_delivery_fee_usd'][i] - \
            fba_commission - df_params['vat_local'][i] - df_params['promotion_discount'][i] - \
            df_params['ppc_cost'][i] - manual_fee - storage_fee
        product_profit_rmb_deal = product_income_deal * df_params['exchange_rate'][i] - df_params['purchase_cost'][i] - \
            df_params['shipping_and_tax'][i]

        # 日常销量
        product_sales = df_params['slope'][i] * prices[i] + df_params['intercept'][i]

        # deal销量
        product_sales_deal = df_params['deal_slope'][i] * prices[i] + df_params['deal_intercept'][i]

        # 总利润
        total_profit += (30 - deal_days[i]) * product_profit_rmb * product_sales \
            + deal_days[i] * product_profit_rmb_deal * product_sales_deal

    return -total_profit  # 由于minimize函数是求最小值，所以返回负的总利润

# 价格区间 [30, 40]
bounds1 = df_params[['price_lower_bound', 'price_upper_bound']].values
bounds1 = [(i[0], i[1]) for i in bounds1]
bounds2 = [(10,10)] + [(0, 10)] * (n_product-1)
bounds = bounds1 + bounds2

# 初始价格
initial_prices1 = np.array([35] * n_product)
initial_prices2 = np.array([5] * n_product)
initial_prices = np.concatenate([initial_prices1, initial_prices2])

# 使用minimize函数求解
result = minimize(total_profit, initial_prices, bounds=bounds, method='L-BFGS-B')

# 最优价格
optimal_prices = result.x
print("Optimal Prices:", optimal_prices)

Optimal Prices: [34.38125428 40.         43.66803274 37.05827994 37.67757197 39.50637623
 42.65746247 36.33735078 42.19277234 45.63308705 44.70175476 46.65832994
 41.10507809 38.70232233 36.51239747 42.29641188 39.44393697 53.87498815
 10.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.         10.        ]


In [9]:
result.success

True

## GA

In [151]:
import numpy as np
from sko.GA import GA, GA_TSP


ga = GA(total_profit, n_dim=36, size_pop=50, max_iter=800, precision=1, lb=[i[0] for i in bounds], ub=[i[1] for i in bounds])
best_x, best_y = ga.run()

best_x, best_y

/Users/adamzh0u/anaconda3/envs/py310/lib/python3.10/site-packages/sko/GA.py:190: RuntimeWarning: invalid value encountered in divide
  return (b * mask).sum(axis=1) / mask.sum()


(array([35., 35., 44., 35., 42., 43., 44., 45., 46., 36., 48., 38., 39.,
        39., 42., 34., 36., 54., nan,  6.,  4.,  9.,  7.,  8., 10., 10.,
         6., 10.,  1.,  8.,  6., 10.,  4.,  0., 10., 10.]),
 array([nan]))

In [153]:
dealx = best_x[-n_product:]
dealx[0] = 10
optimized_pricesx = best_x[:-n_product]
# optimized_pricesx = [round(i, 0) for i in optimized_pricesx]

# optimized_pricesx[1] =40
# dealx[1] = 0
dftr = result_process(dft, optimized_pricesx, dealx)
dftr.month_profit_rmb.sum()

139342.01516399998